In [12]:
import age
from age.graph_builder import *
from age.gen.ageParser import *
import antlr4

class TestResultHandler(ResultHandler):
    def __init__(self):
        self.lexer = ageLexer()
        stream = CommonTokenStream(self.lexer)
        self.parser = ageParser(stream)
        self.visitor = ResultVisitor()

    def handleRow(self, graph, rawString):
        rawstream = InputStream(rawString)
        print(self.lexer, rawstream)
        print(self.lexer.inputStream)
        self.lexer.inputStream = rawstream
        self.parser.reset()
        tree = self.parser.ageout()
        parsed = tree.accept(self.visitor)
        print(parsed)

        
        
ag = age.newAge(graph="my_graph", host="172.17.0.3", port="5432", dbname="postgres", user="postgres", password="asdf11")
cur = ag.queryCypher("MATCH (n) RETURN n")
ag.buildGraph(cur , resultHandler=TestResultHandler())

print("----------------")
cur = ag.queryCypher("MATCH p=()-[:has]-() RETURN *",  ("has",))
ag.buildGraph(cur, resultHandler=age.rawPrinter)


<age.gen.ageLexer.ageLexer object at 0x7f20a8891940> {"id": 844424930131970, "label": "Part", "properties": {"part_num": "345"}}::vertex
None
None
<age.gen.ageLexer.ageLexer object at 0x7f20a8891940> {"id": 844424930131971, "label": "Part", "properties": {"part_num": "456"}}::vertex
{"id": 844424930131970, "label": "Part", "properties": {"part_num": "345"}}::vertex
None
<age.gen.ageLexer.ageLexer object at 0x7f20a8891940> {"id": 844424930131972, "label": "Part", "properties": {"part_num": "789"}}::vertex
{"id": 844424930131971, "label": "Part", "properties": {"part_num": "456"}}::vertex
None
<age.gen.ageLexer.ageLexer object at 0x7f20a8891940> {"id": 844424930131973, "label": "Part", "properties": {"part_num": "1230"}}::vertex
{"id": 844424930131972, "label": "Part", "properties": {"part_num": "789"}}::vertex
None
<age.gen.ageLexer.ageLexer object at 0x7f20a8891940> {"id": 844424930131974, "label": "Part", "properties": {"part_num": "3450"}}::vertex
{"id": 844424930131973, "label": "Pa